### Определение категории контрагента по данным из различных источников
- 1с, 
- VTBL_MSCRM.dbo.AccountBase   
- MDM_Main.dbo.egrip_regdata (egrul_regdata)  
- dataset_local_application_inf_ssf.01.01.2022.pkl
- DM.BEHAVIOR_MODEL_DAILY

Входные данные 
- data/external/external_pkl/target_dataset.pkl   
- data/interm/external_pkl/portfolio_new_version.pkl  
- data/interm/external_pkl/all_counteragent_1c.pkl
- data/interm/external_pkl/crm_account.pkl  
- data/interm/external_pkl/regdata_from_mdm.pkl  
- data/interm/external_pkl/clients_bmd.pkl

Выходной файл data/interm/accounts_status.pkl

In [1]:
import pandas as pd

In [2]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки
%run ../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init_mdm.ipynb
%run ../preprocessing/preprocessing_target.ipynb
%run ../ML_functions.ipynb
%run ../auxiliary/plots.ipynb

In [3]:
# сюда таблицы будем кидать, для отчетов
name = pre + "data/interm/auxilary/pre_accounts.xlsx"
writer = ExcelWriter(name)

In [4]:
name = pre + "data/interm/external_pkl/target_dataset.pkl"
target_data = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_data))
# target_data.head(1)

85798


In [5]:
name = pre + "data/external/portfolio_1c.pkl"

keep_cols = [ "Код договора лизинга", "ИНН",  "Контрагент", "Номер договора", "Дата договора", "Дата лизинга", "Дата приложения от", 
        "Срок приложения до", "Статус приложения", 
            "Дата окончания лизинга фактическая", "Завершение_агрегированное", "ЗаводскойНомер", "new_cartypeName", 'Первичный', 'Договор_с_цессией']

file_portfolio_raw = pd.read_pickle(name, compression={'method':'zip'}).sort_values(by=["Номер договора", "Дата приложения от"])

print("Прочитанный размер файла портфеля {} - {}".format(name, file_portfolio_raw.shape))
print("Дата актуальности портфеля 1с {}".format(file_portfolio_raw.loc[0, 'Дата актуальности'].strftime("%Y-%m-%d")))

Прочитанный размер файла портфеля ../../data/external/portfolio_1c.pkl - (365081, 44)
Дата актуальности портфеля 1с 2023-07-06


In [6]:
name = pre + "data/external/all_counteragent_1c.pkl"
all_counteragent_1c = pd.read_pickle(name, compression={'method':'zip'})

print("Размер считанных данных контрагентов из 1c - {}".format(all_counteragent_1c.shape))

# all_counteragent_1c.head(1)

Размер считанных данных контрагентов из 1c - (193775, 6)


In [7]:
name = pre + "data/interm/external_pkl/crm_account.pkl"
data_accounts = pd.read_pickle(name, compression={'method':'zip'})

data_accounts = data_accounts.drop_duplicates(subset='NEW_INN')

print("Размер считанных данных контрагентов из crm - {}".format(data_accounts.shape))
num, duplicates = identify_duplicates(data_accounts, 'NEW_INN')
Check_dupl(num, duplicates)
# data_accounts.head(1)

Размер считанных данных контрагентов из crm - (722812, 10)


In [8]:
name = pre + "data/interm/external_pkl/regdata_from_mdm.pkl"
regdata_from_mdm = pd.read_pickle(name, compression={'method':'zip'})
regdata_from_mdm = regdata_from_mdm.sort_values(by='new_dissolutiondate').drop_duplicates(subset='new_inn').reset_index(drop=True)

print("Всего выгружено контрагентов из regdata mdm  - {}".format(regdata_from_mdm.shape))

num, duplicates = identify_duplicates(regdata_from_mdm, 'new_inn')
Check_dupl(num, duplicates)

# regdata_from_mdm.head(1)

Всего выгружено контрагентов из regdata mdm  - (42161, 4)


In [9]:
name = pre + "data/interm/external_pkl/clients_bmd.pkl"
clients_bmd = pd.read_pickle(name, compression={'method':'zip'})

print("Размер считанных данных контрагентов из DM.BEHAVIOR_MODEL_DAILY ", clients_bmd.shape)
# clients_bmd.head(1)

Размер считанных данных контрагентов из DM.BEHAVIOR_MODEL_DAILY  (93075, 1)


## Анализ статуса нашиз клиентов по данным 1c

In [10]:
file_portfolio_raw['Статус приложения'] = file_portfolio_raw['Статус приложения'].fillna('Не было передачи в лизинг')
file_portfolio_raw['Статус приложения'] = pd.Categorical(file_portfolio_raw['Статус приложения'], 
                                            categories=['Проект (принятие к управленческому учету)', 'Не было передачи в лизинг', 
                                            'Закрыт', 'Страховой случай', 'Расторгнут в связи со страховым случаем', 'Расторгнут', 
                                            'Подписан (вступил в силу)'], 
                                            ordered=True)

file_portfolio_raw = file_portfolio_raw.sort_values(by=['ИНН', 'Статус приложения', 'Дата приложения от', 'Срок приложения до']).reset_index(drop=True)                                

In [11]:
inn_1c = file_portfolio_raw[['ИНН']].drop_duplicates().reset_index(drop=True)
# inn_1c.head()

смотрим на статуc нашего ИНН отдельно у тех, у кого был лизинг, по данным портфеля

In [12]:
mask = (file_portfolio_raw['Дата лизинга'].notnull())
port_leasing = file_portfolio_raw[mask].copy()

mask = (port_leasing['Статус приложения'].isin(['Подписан (вступил в силу)']))
port_leasing.loc[mask, 'auxilary'] = 1
port_leasing.loc[~mask, 'auxilary'] = 0
list_inn = port_leasing[port_leasing['auxilary']==1]['ИНН'].to_list()
mask = inn_1c['ИНН'].isin(list_inn)
inn_1c.loc[mask, 'Статус_контрагента'] = 'Есть действующий договор(ы)'


mask = (port_leasing['Статус приложения'].isin(['Расторгнут в связи со страховым случаем', 'Расторгнут']))
port_leasing.loc[mask, 'auxilary'] = 1
port_leasing.loc[~mask, 'auxilary'] = 0
list_inn = port_leasing[port_leasing['auxilary']==1]['ИНН'].to_list()
mask = (inn_1c['ИНН'].isin(list_inn))
mask_nostatus = (inn_1c['Статус_контрагента'].isnull())
inn_1c.loc[mask & mask_nostatus, 'Статус_контрагента'] = 'Клиент в прошлом (расторжение)'


mask = (port_leasing['Статус приложения'].isin(['Закрыт', 'Страховой случай']))
port_leasing.loc[mask, 'auxilary'] = 1
port_leasing.loc[~mask, 'auxilary'] = 0
list_inn = port_leasing[port_leasing['auxilary']==1]['ИНН'].to_list()
mask = (inn_1c['ИНН'].isin(list_inn))
mask_nostatus = (inn_1c['Статус_контрагента'].isnull())
inn_1c.loc[mask & mask_nostatus, 'Статус_контрагента'] = 'Клиент в прошлом'

mask = (port_leasing['Статус приложения'].isin(['Проект (принятие к управленческому учету)', 'Не было передачи в лизинг']))
port_leasing.loc[mask, 'auxilary'] = 1
port_leasing.loc[~mask, 'auxilary'] = 0
list_inn = port_leasing[port_leasing['auxilary']==1]['ИНН'].to_list()
mask = (inn_1c['ИНН'].isin(list_inn))
mask_nostatus = (inn_1c['Статус_контрагента'].isnull())
inn_1c.loc[mask & mask_nostatus, 'Статус_контрагента'] = 'Не было передачи в лизинг'


inn_1c['Статус_контрагента'] = inn_1c['Статус_контрагента'].fillna('Не было передачи в лизинг')
# inn_1c.head()

In [13]:
print('Не было передачи в лизинг у {}'.format(len(inn_1c[inn_1c['Статус_контрагента'] == 'Не было передачи в лизинг'])))

Не было передачи в лизинг у 30185


In [14]:
inn_1c = inn_1c.reset_index()
# inn_1c.head()

проверка, что может быть расторжен, но это у договора, по которому не было лизинга

In [15]:
df = pd.pivot_table(inn_1c, index='Статус_контрагента',  values='index', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'index':'Количество ИНН'})
df['Доля, %'] = round(df['Количество ИНН']/df['Количество ИНН']['All']*100,2)
# df

### Дополнительно смотрим, были какие-либо другие договоры помимо лизинга

In [16]:
all_counteragent_1c['хлВидДоговора'] = pd.Categorical(all_counteragent_1c['хлВидДоговора'], 
                                            categories=['Договор обратного выкупа', 
                                            'С покупателем', 'Договор купли-продажи','Услуги оказанные', 'Лизинга', 'Договор цессии'], ordered=True)

mask = (all_counteragent_1c['хлВидДоговора']!='Лизинга')
all_contracts_1c = all_counteragent_1c[mask]

all_contracts_1c = all_contracts_1c.sort_values(by=['ИНН', 'хлВидДоговора']).drop_duplicates(subset='ИНН', keep='first').reset_index(drop=True)

repl = {'Лизинга':'Был договор лизинга', 'С покупателем':'Был договор с покупателем', 'Услуги оказанные':'Был договор по услугам', 
        'Договор купли-продажи':'Был договор купли-продажи', 'Договор обратного выкупа':'Был договор обратного выкупа',
       'Договор цессии':'Был договор цессии'}
all_contracts_1c['ВидДоговора'] = all_contracts_1c['хлВидДоговора'].replace(repl)

num, duplicates = identify_duplicates(all_contracts_1c, 'ИНН')
Check_dupl(num, duplicates)
all_contracts_1c['ВидДоговора'].unique()

array(['Был договор с покупателем', 'Был договор по услугам',
       'Был договор купли-продажи', 'Был договор обратного выкупа',
       'Был договор цессии'], dtype=object)

In [17]:
status_1c = pd.merge(inn_1c, all_contracts_1c, how='outer', on='ИНН')
status_1c.drop(columns='index', inplace=True)
status_1c = status_1c.reset_index()

status_1c['Статус_контрагента'] = status_1c['Статус_контрагента'].combine_first(status_1c['ВидДоговора'])

mask = (status_1c['Статус_контрагента']=='Не было передачи в лизинг')
mask_other = (status_1c['ВидДоговора'].notnull())
print("Не было передачи в лизинг, но был другой тип договора у {}".format(len(status_1c[mask & mask_other])))
status_1c.loc[mask & mask_other, 'Статус_контрагента'] = status_1c.loc[mask & mask_other, 'ВидДоговора']

# status_1c.head(1)

Не было передачи в лизинг, но был другой тип договора у 150


In [18]:
df = pd.pivot_table(status_1c, index='Статус_контрагента',  values='index', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'index':'Количество ИНН'})
df['Доля, %'] = round(df['Количество ИНН']/df['Количество ИНН']['All']*100,2)
# df

## Статусы контрагентов по данным VTBL_MSCRM.dbo.AccountBase

In [19]:
data_accounts = data_accounts[data_accounts['NEW_INN'].notnull()].reset_index(drop=True)

In [20]:
# data_accounts.head(1)

In [21]:
data_accounts['4reliability_group'] = data_accounts['new_4reliability_group'].notnull()
data_accounts['black_list_date'] = data_accounts['new_black_list_date'].notnull()

In [22]:
df = pd.pivot_table(data_accounts, index='4reliability_group',  values='NEW_INN', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'NEW_INN':'Количество ИНН'})
df['Доля, %'] = round(df['Количество ИНН']/df['Количество ИНН']['All']*100,2)
# df

In [23]:
df = pd.pivot_table(data_accounts, index='black_list_date',  values='NEW_INN', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'NEW_INN':'Количество ИНН'})
df['Доля, %'] = round(df['Количество ИНН']/df['Количество ИНН']['All']*100,2)
# df

In [24]:
df = pd.pivot_table(data_accounts, index=['4reliability_group','black_list_date'],  values='NEW_INN', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'NEW_INN':'Количество ИНН'}).reset_index()
all = df.loc[df['4reliability_group']=='All',  'Количество ИНН'].values
df['Доля, %'] = round(df['Количество ИНН']/all*100,2)
# df.set_index(['4reliability_group','black_list_date'])

In [25]:
df = pd.pivot_table(data_accounts, index='StatusCodeName',  values='NEW_INN', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'NEW_INN':'Количество ИНН'})
df['Доля, %'] = round(df['Количество ИНН']/df['Количество ИНН']['All']*100,2)
# df

In [26]:
df = pd.pivot_table(data_accounts, index='SupTypeName',  values='NEW_INN', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'NEW_INN':'Количество ИНН'})
df['Доля, %'] = round(df['Количество ИНН']/df['Количество ИНН']['All']*100,2)
# df

In [27]:
df = pd.pivot_table(data_accounts, index=['StatusCodeName','SupTypeName'],  values='NEW_INN', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'NEW_INN':'Количество ИНН'}).reset_index()
all = df.loc[df['StatusCodeName']=='All',  'Количество ИНН'].values
df['Доля, %'] = round(df['Количество ИНН']/all*100,2)
# df.set_index(['StatusCodeName','SupTypeName'])

## Статусы клиентов по target данным

In [28]:
target_data = target_data[target_data['Попадает в выборку'].isin(['обе модели', 'только фрод'])]

mask = (target_data['Попадает в выборку']=='обе модели')
target_data.loc[mask, 'Метка дефолта'] = target_data.loc[mask, ['Метка кред дефолта', 'Метка фрод дефолта']].max(axis=1)

mask = (target_data['Попадает в выборку']=='только фрод')
target_data.loc[mask, 'Метка дефолта'] = target_data.loc[mask, 'Метка фрод дефолта']

# берем по контрагенту метку максимальную
target_data['Метка дефолта'] =  target_data.groupby(['ИНН контрагента'])['Метка дефолта'].transform(max)

target_data = target_data.drop_duplicates(subset='ИНН контрагента')
target_data = target_data[target_data['ИНН контрагента'].notnull()].reset_index(drop=True)

In [29]:
df = pd.pivot_table(target_data, index=['Метка дефолта'],  values='ИНН контрагента', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'ИНН контрагента':'Количество ИНН'})
df['Доля, %'] = round(df['Количество ИНН']/df['Количество ИНН']['All']*100,2)
# df

## Статусы контрагентов по данным regdata from mdm 
[MDM_Main].[dbo].[egrul_regdata] и [MDM_Main].[dbo].[egrip_regdata]

In [30]:
# regdata_from_mdm.head()

In [31]:
df = pd.pivot_table(regdata_from_mdm, index='new_typeofdissolution',  values='new_inn', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'new_inn':'Количество ИНН'})
df['Доля, %'] = round(df['Количество ИНН']/df['Количество ИНН']['All']*100,2)
df

,Количество ИНН,"Доля, %"
new_typeofdissolution,,
Государственная регистрация физического лица в качестве индивидуального предпринимателя признана недействительной в соответствии с пунктом 4 статьи 22.1 Федерального закона от 8 августа 2001 года № 129-ФЗ,2,0.01
Индивидуальный предприниматель прекратил деятельность в связи с принятием им соответствующего решения,5953,18.52
Индивидуальный предприниматель прекратил деятельность в связи со смертью,283,0.88
Исключение из ЕГРЮЛ недействующего юридического лица,6323,19.67
"Исключение из ЕГРЮЛ юридического лица в связи наличием в ЕГРЮЛ сведений о нем, в отношении которых внесена запись о недостоверности",8902,27.69
Исключение из ЕГРЮЛ юридического лица в связи с невозможностью его ликвидации,3,0.01
Крестьянское (фермерское) хозяйство прекратило деятельность по решению членов крестьянского (фермерского) хозяйства,222,0.69
Крестьянское (фермерское) хозяйство утратило государственную регистрацию на основании статьи 3 ФЗ от 23.06.2003 №76-ФЗ,1,0.00
Ликвидация некоммерческой организации по решению суда,16,0.05


In [32]:
regdata = regdata_from_mdm.copy()
regdata['new_accounstatus'] = regdata['new_accounstatus'].fillna('')

list_good = ['реорганизац', 'нахождени', 'уменьшения уставного капитала']
reason_good  =r'(?P<good>' + "|".join(i for i in list_good) + ')'
regdata = regdata.join(regdata['new_accounstatus'].str.extract(reason_good, re.IGNORECASE))
mask_good = (regdata['good'].notnull())
regdata.loc[mask_good, 'status_good'] = True 
regdata.loc[~mask_good, 'status_good'] = False 

regdata.drop(columns = ['good'], inplace=True)

mask_bad = (regdata['new_accounstatus'].str.contains('прекратит деятельность', re.IGNORECASE))
regdata.loc[mask_bad, 'status_good'] = False

In [33]:
df = pd.pivot_table(regdata, index=['new_accounstatus', 'status_good'],  values='new_inn', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'new_inn':'Количество ИНН'})
# df

In [34]:
mask = (regdata['new_typeofdissolution'].notnull())
mask_date = (regdata['new_dissolutiondate'].notnull())
mask_bad = (regdata['status_good']== False)

# display_side_by_side(regdata[(mask | mask_date) & (~mask_bad)])

In [35]:
"""  будем считать недействительными организациями, там где заполнена дата прекращения деятельности 
    или тип прекращения задан 
    либо плохой статус стоит
"""
mask = (regdata['new_typeofdissolution'].notnull())
mask_date = (regdata['new_dissolutiondate'].notnull())
mask_bad = (regdata['status_good']== False)

regdata_bad = regdata[mask | mask_date | mask_bad].reset_index(drop=True)

regdata_bad['status_mdm'] = 'Недействующая организация'

print("Недействующих организаций по данным mdm ", len(regdata_bad))

df = pd.pivot_table(regdata_bad, index='new_accounstatus',  values='new_inn', aggfunc='count', margins=True, fill_value= 0)\
    .rename(columns={'new_inn':'Количество ИНН'})
df['Доля, %'] = round(df['Количество ИНН']/df['Количество ИНН']['All']*100,2)
# df

Недействующих организаций по данным mdm  40307


## Определим общий статус по самому консервативному сценарию

In [36]:
df = pd.merge(status_1c[['ИНН', 'Статус_контрагента']], 
              data_accounts[['NEW_INN', 'StatusCodeName','SupTypeName', '4reliability_group', 'black_list_date']], 
              how='outer', left_on=['ИНН'], right_on=['NEW_INN'])

df['inn'] = df['ИНН'].combine_first(df['NEW_INN'])


gen_status = pd.merge(df, target_data[['ИНН контрагента', 'Метка дефолта']], how='outer', left_on=['inn'], right_on=['ИНН контрагента']).reset_index(drop=True)
gen_status['inn'] = gen_status['inn'].combine_first(gen_status['ИНН контрагента'])

gen_status = pd.merge(gen_status, regdata_bad[['new_inn', 'status_mdm', 'new_typeofdissolution',	'new_dissolutiondate',	'new_accounstatus']], 
                        how='outer', left_on=['inn'], right_on=['new_inn']).reset_index(drop=True)
gen_status['inn'] = gen_status['inn'].combine_first(gen_status['new_inn'])


# определяем дубликаты по ID
num, duplicates = identify_duplicates(gen_status, 'inn')
Check_dupl(num, duplicates)

gen_status['inn'].isnull().sum()

0

In [37]:
# gen_status.head(1)

In [38]:
print('Статус_контрагента: ', gen_status['Статус_контрагента'].unique())

print('StatusCodeName: ', gen_status['StatusCodeName'].unique())
print('SupTypeName: ', gen_status['SupTypeName'].unique())
print('4reliability_group: ', gen_status['4reliability_group'].unique())
print('black_list_date: ', gen_status['black_list_date'].unique())

print('Метка дефолта: ', gen_status['Метка дефолта'].unique())

print('status_mdm: ', gen_status['status_mdm'].unique())

Статус_контрагента:  ['Есть действующий договор(ы)' 'Клиент в прошлом'
 'Клиент в прошлом (расторжение)' 'Не было передачи в лизинг'
 'Был договор с покупателем' 'Был договор купли-продажи'
 'Был договор цессии' 'Был договор по услугам' nan]
StatusCodeName:  ['Активный' 'Неактивный' nan 'Отказ УОБ']
SupTypeName:  ['В работе' 'Недействующая организация' 'Черный список' nan
 'Работа не ведется']
4reliability_group:  [False True nan]
black_list_date:  [False True nan]
Метка дефолта:  [nan  0.  1.]
status_mdm:  [nan 'Недействующая организация']


In [39]:
mask = (gen_status['status_mdm']=='Недействующая организация')
gen_status.loc[mask, 'Статус_общ'] = 'Недействующая организация'

mask_null = (gen_status['Статус_общ'].isnull())
mask = (gen_status['SupTypeName'].isin(['Черный список', 'Недействующая организация']))
gen_status.loc[mask & mask_null, 'Статус_общ'] = gen_status.loc[mask, 'SupTypeName']

mask_null = (gen_status['Статус_общ'].isnull())
mask = (gen_status['4reliability_group'])
gen_status.loc[mask & mask_null, 'Статус_общ'] = 'Четвёртая группа надежности'

mask_null = (gen_status['Статус_общ'].isnull())
mask = (gen_status['black_list_date'])
gen_status.loc[mask & mask_null, 'Статус_общ'] = 'Черный список'

mask_null = (gen_status['Статус_общ'].isnull())
mask = (gen_status['StatusCodeName']=='Отказ УОБ')
gen_status.loc[mask & mask_null, 'Статус_общ'] = 'Отказ УОБ'

mask_null = (gen_status['Статус_общ'].isnull())
mask = (gen_status['Статус_контрагента']=='Клиент в прошлом (расторжение)')
gen_status.loc[mask & mask_null, 'Статус_общ'] = gen_status.loc[mask & mask_null, 'Статус_контрагента']

mask_null = (gen_status['Статус_общ'].isnull())
mask = (gen_status['Метка дефолта']==1)
gen_status.loc[mask & mask_null, 'Статус_общ'] = 'Метка дефолта'

mask_null = (gen_status['Статус_общ'].isnull())
mask = (gen_status['Статус_контрагента'].notnull())
gen_status.loc[mask & mask_null, 'Статус_общ'] = gen_status.loc[mask & mask_null, 'Статус_контрагента']

mask_null = (gen_status['Статус_общ'].isnull())
gen_status.loc[mask_null, 'Статус_общ'] = 'Не является нашим клиентом'

In [40]:
df = pd.pivot_table(gen_status, index='Статус_общ',  values='inn', aggfunc='count', 
                margins=True, fill_value= 0).rename(columns={'inn':'Количество ИНН'})
# df

In [41]:
df = pd.pivot_table(gen_status, index=['Статус_общ', 'Статус_контрагента'],  values='inn', aggfunc='count', 
                margins=True, fill_value= 0).rename(columns={'inn':'Количество ИНН'})
# df

In [42]:
#  это ИП прекратили действие
mask = (gen_status['Статус_общ']=='Недействующая организация')
mask_gen = (gen_status['Статус_контрагента']=='Есть действующий договор(ы)')
# gen_status[mask&mask_gen].head()

### смотрим, есть ли они в bmd

In [43]:
clients = pd.merge(gen_status, clients_bmd, how='left', left_on='inn', right_on='INN_IN_BMD')

mask = (clients['INN_IN_BMD'].notnull())
clients.loc[mask, 'is_in_bmd'] = True
clients.loc[~mask, 'is_in_bmd'] = False


In [44]:
clients[['inn', 'Статус_общ', 'Статус_контрагента', 'is_in_bmd']].to_pickle(pre+"data/interm/accounts_status.pkl", compression={'method':'zip'})
status_1c[['ИНН', 'Статус_контрагента']].to_pickle(pre+"data/interm/accounts_status1c.pkl", compression={'method':'zip'})

In [45]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save()